# Tutorial: Classification Fine-tuning

Let's walk through a quick example of fine-tuning the LM weights within a DSPy program. We'll apply to a simple 77-way classification task.

Our finetuned program will use a tiny `Llama-3.2-1B` language model, hosted locally on your GPU. To make this more interesting, we'll assume that (i) we don't have **any training labels** but (ii) we have 500 unlabeled training examples.

### Install dependencies and download data

Install the latest DSPy via `pip install -U --pre dspy` and follow along. This tutorial depends on DSPy 2.6.0 (pre-release).

This tutorial requires a local GPU at the moment for inference, though we plan to support ollama serving for finetuned models as well.

You will also need the following dependencies:

```shell
> pip install "sglang[all]"; pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/
> pip install -U torch transformers accelerate trl peft
```

### Dataset

For this tutorial, we will use the Banking77 dataset.

In [ ]:
import dspy
import random
from dspy.datasets import DataLoader
from datasets import load_dataset

# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
]

random.Random(0).shuffle(raw_data)

This dataset has 77 different categories for classification. Let's review some of them.

In [3]:
len(CLASSES), CLASSES[:10]

(77,
 ['activate_my_card',
  'age_limit',
  'apple_pay_or_google_pay',
  'atm_support',
  'automatic_top_up',
  'balance_not_updated_after_bank_transfer',
  'balance_not_updated_after_cheque_or_cash_deposit',
  'beneficiary_not_allowed',
  'cancel_transfer',
  'card_about_to_expire'])

Let us sample 500 (unlabeled) queries from Banking77. We'll use these for our bootstrapped finetuning.

In [4]:
unlabeled_trainset = [dspy.Example(text=x.text).with_inputs("text") for x in raw_data[:500]]

unlabeled_trainset[0]

Example({'text': 'What if there is an error on the exchange rate?'}) (input_keys={'text'})

### DSPy program

Let's say that we want a program that takes the `text` and reasons step by step and then selects one of the classes from Banking77.

Note that this is meant mainly for illustration, or for cases where you want to inspect the model's reasoning, e.g. for a small degree of explainability. In other words, this type of task is not necessarily likely to benefit very much from explicit reasoning.

In [ ]:
from typing import Literal

classify = dspy.ChainOfThought(f"text -> label: Literal{CLASSES}")

### Bootstrapped finetuning

There are many ways to go about this, e.g. allowing the model to teach itself or using inference-time compute (e.g., ensembling) to identify cases of high confidence without labels.

Perhaps the simplest is to use a model that we'd expect can do a reasonable job at this task as a teacher of reasoning and classification, and to distill that to our small model. All of these patterns can be expressed in a handful of lines of code.

Let's set up the tiny `Llama-3.2-1B-Instruct` as a student LM. We'll use GPT-4o-mini as a teacher LM.

In [ ]:
from dspy.clients.lm_local import LocalProvider

student_lm_name = "meta-llama/Llama-3.2-1B-Instruct"
student_lm = dspy.LM(model=f"openai/local:{student_lm_name}", provider=LocalProvider(), max_tokens=2000)
teacher_lm = dspy.LM('openai/gpt-4o-mini', max_tokens=3000)

Now, let's assign classifiers to our LMs.

In [ ]:
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

teacher_classify = classify.deepcopy()
teacher_classify.set_lm(teacher_lm)

Let's now launch the bootstrapped finetuning. The word "bootstrapped" here means that the program itself will be invoked on the training inputs and the resulting traces seen over all modules will be recorded and used for finetuning. This is the weight-optimizing variant of the various BootstrapFewShot methods in DSPy.

On every question in the (unlabeled) training set, this will invoke the teacher program, which will produce reasoning and select a class. This will be traced and then constitute a training set for all modules (in this case, just the one CoT module) in the student program.

Note: If you have labels, you can pass `metric` to the constructor of `BootstrapFinetune`. If you want to apply this in practice, you can pass `train_kwargs` to the constructor to control local LM training settings: `device`, `use_peft`, `num_train_epochs`, `per_device_train_batch_size`, `gradient_accumulation_steps`, `learning_rate`, `max_seq_length`, `packing`, `bf16`, and `output_dir`.

In [ ]:
dspy.settings.experimental = True  # Fine-tuning is still experimental, so we need to enable it first!

optimizer = dspy.BootstrapFinetune(num_threads=16)  # if you *do* have labels, pass metric=your_metric here!
classify_ft = optimizer.compile(student_classify, teacher=teacher_classify, trainset=unlabeled_trainset)

Since this is a local model, we need to explicitly launch it.

In [ ]:
classify_ft.get_lm().launch()

### Validating the finetuned program

Let's now figure out if this was successful. We can ask the system one question and inspect its behavior.

In [12]:
classify_ft(text="I didn't receive my money earlier and it says the transaction is still in progress. Can you fix it?")

Prediction(
    reasoning='The user is inquiring about a discrepancy in the amount of money they did not receive earlier, which suggests they are experiencing an issue related to the amount of money that was not successfully processed or received. This aligns with the label for issues regarding the amount of money not received during a transaction.',
    label='wrong_amount_of_cash_received'
)

We could also get a small set of gold labels and see if the system can generalize to unseen queries.

In [13]:
devset = raw_data[500:600]
devset[0]

Example({'text': 'Which fiat currencies do you currently support? Will this change in this future?', 'label': 'fiat_currency_support'}) (input_keys={'text'})

Let's define an evaluator on this small dev set, where the metric ignores the reasoning and checks that the label is exactly correct.

In [14]:
metric = (lambda x, y, trace=None: x.label == y.label)
evaluate = dspy.Evaluate(devset=devset, metric=metric, display_progress=True, display_table=5, num_threads=16)

Now, let's evaluate the finetuned 1B classifier.

In [15]:
evaluate(classify_ft)

Average Metric: 64.00 / 100 (64.0%): 100%|██████████| 100/100 [00:34<00:00,  2.92it/s]

2025/01/31 15:20:06 INFO dspy.evaluate.evaluate: Average Metric: 64 / 100 (64.0%)


,text,example_label,reasoning,pred_label,<lambda>
0,Which fiat currencies do you currently support? Will this change i...,fiat_currency_support,The user is inquiring about the fiat currencies that are currently...,fiat_currency_support,✔️ [True]
1,I didn't receive my money earlier and it says the transaction is s...,pending_cash_withdrawal,The user is inquiring about a discrepancy in the amount of money t...,wrong_amount_of_cash_received,
2,what currencies do you accept?,fiat_currency_support,"The user is inquiring about the currencies that are accepted, whic...",fiat_currency_support,✔️ [True]
3,Where can I find your exchange rates?,exchange_rate,"The user is inquiring about where to find exchange rates, which re...",exchange_rate,✔️ [True]
4,why hasnt my card come in yet?,card_arrival,"The user is inquiring about the status of their card, specifically...",card_arrival,✔️ [True]


64.0

Not bad, given that we started with no labels of the task. Even though we have no labels, you can use various strategies to boost the quality of the bootstrapped training data.

To try that next, let's free our GPU memory by killing the finetuned LM.

In [ ]:
classify_ft.get_lm().kill()

### Bootstrapped finetuning against a metric

If you have labels, you can generally boost this by a large margin. To do so, you can pass a `metric` to BootstrapFinetune, which it will use for filtering the trajectories over your program before it builds the finetuning data.

In [ ]:
optimizer = dspy.BootstrapFinetune(num_threads=16, metric=metric)
classify_ft = optimizer.compile(student_classify, teacher=teacher_classify, trainset=raw_data[:500])

Let's now launch and evaluate this.

In [ ]:
classify_ft.get_lm().launch()

In [19]:
evaluate(classify_ft)

  0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 83.00 / 100 (83.0%): 100%|██████████| 100/100 [00:30<00:00,  3.25it/s]

2025/01/31 15:29:15 INFO dspy.evaluate.evaluate: Average Metric: 83 / 100 (83.0%)


,text,example_label,reasoning,pred_label,<lambda>
0,Which fiat currencies do you currently support? Will this change i...,fiat_currency_support,"The question is about the types of fiat currencies supported, whic...",fiat_currency_support,✔️ [True]
1,I didn't receive my money earlier and it says the transaction is s...,pending_cash_withdrawal,The user is experiencing an issue with not receiving their money e...,pending_cash_withdrawal,✔️ [True]
2,what currencies do you accept?,fiat_currency_support,"The user is inquiring about the currencies that are supported, whi...",fiat_currency_support,✔️ [True]
3,Where can I find your exchange rates?,exchange_rate,"The user is inquiring about the location of the exchange rates, wh...",exchange_rate,✔️ [True]
4,why hasnt my card come in yet?,card_arrival,"The user is inquiring about the status of their card delivery, ind...",card_arrival,✔️ [True]


83.0

That's quite a bit better, given just 500 labels. In fact, it seems to be a lot stronger than the teacher LM gets out of the box!

In [20]:
evaluate(teacher_classify)

  0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 71.00 / 100 (71.0%): 100%|██████████| 100/100 [00:16<00:00,  6.13it/s]

2025/01/31 15:29:37 INFO dspy.evaluate.evaluate: Average Metric: 71 / 100 (71.0%)


,text,example_label,reasoning,pred_label,<lambda>
0,Which fiat currencies do you currently support? Will this change i...,fiat_currency_support,The user is inquiring about the fiat currencies supported by the s...,fiat_currency_support,✔️ [True]
1,I didn't receive my money earlier and it says the transaction is s...,pending_cash_withdrawal,The user is experiencing an issue with a transaction that is still...,pending_transfer,
2,what currencies do you accept?,fiat_currency_support,"The user is inquiring about the types of currencies accepted, whic...",fiat_currency_support,✔️ [True]
3,Where can I find your exchange rates?,exchange_rate,"The user is inquiring about where to find exchange rates, which re...",exchange_rate,✔️ [True]
4,why hasnt my card come in yet?,card_arrival,"The user is inquiring about the status of their card delivery, whi...",card_arrival,✔️ [True]


71.0

And thanks to bootstrapping, the model learns to apply our modules to get the right label, in this case, reasoning explicitly:

In [21]:
classify_ft(text="why hasnt my card come in yet?")
dspy.inspect_history()





[2025-01-31T15:29:47.054096]

System message:

Your input fields are:
1. `text` (str)

Your output fields are:
1. `reasoning` (str)
2. `label` (Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_